In [3]:
# O(a + r) time | O(a + r) space
def airportConnections(airports, routes, startingAirport):
    edges, airportIndexMap = buildAdjacencyList(airports, routes)

    # build strongly connected component (SCC) graph
    # O(a + r)
    sccEdges, sccMap = buildSCCAdjacencyList(edges)

    # We need to add a connection to all roots not reachable by start node
    startNode = sccMap[airportIndexMap[startingAirport]]
    numComponents = len(sccEdges)
    incomingEdges = reverseEdges(sccEdges)
    numRoutes = 0
    for i in range(numComponents):
        if i != startNode and len(incomingEdges[i]) == 0:
            numRoutes += 1
    return numRoutes

def buildAdjacencyList(airports, routes):
    n = len(airports)
    airportIndexMap = {}
    for index, airport in enumerate(airports):
        airportIndexMap[airport] = index

    edges = [[] for _ in range(n)]
    for start, dest in routes:
        edges[airportIndexMap[start]].append(airportIndexMap[dest])
    
    return edges, airportIndexMap

def reverseEdges(edges):
    n = len(edges)
    incomingEdges = [[] for _ in range(n)]
    for i in range(n):
        for neighbor in edges[i]:
            incomingEdges[neighbor].append(i)
    return incomingEdges

def buildSCCAdjacencyList(edges):
    """
    Compute strongly connected component (SCC) graph with Kosaraju-Sharir algorithm
    (https://www.coursera.org/learn/algorithms-part2/lecture/fC5Yw?t=184)
    """
    # compute reversed post order on reversed graph
    reversedEdges = reverseEdges(edges)
    reversedPostOrderOnReversedGraph = computeReversedPostOrder(reversedEdges)

    # compute SCC Map by traversing according to reversedPostOrderOnReversedGraph
    sccMap, numSCCs = computeSCCMap(reversedPostOrderOnReversedGraph, edges)

    # build SCC adjacency list
    n = len(edges)
    sccEdges = [set() for _ in range(numSCCs)]
    for i in range(n):
        for neighbor in edges[i]:
            if sccMap[i] != sccMap[neighbor]:
                sccEdges[sccMap[i]].add(sccMap[neighbor])
    for i in range(numSCCs):
        sccEdges[i] = list(sccEdges[i])
    
    return sccEdges, sccMap

def computeSCCMap(reversedPostOrderOnReversedGraph, edges):
    n = len(reversedPostOrderOnReversedGraph)
    sccMap = [-1] * n
    numSCCs = 0
    for i in range(n):
        node = reversedPostOrderOnReversedGraph[i]
        if sccMap[node] == -1:
            sccMapDFS(node, edges, numSCCs, sccMap)
            numSCCs += 1
    return sccMap, numSCCs

def sccMapDFS(node, edges, sccNumber, sccMap):
    sccMap[node] = sccNumber
    for neighbor in edges[node]:
        if sccMap[neighbor] == -1:
            sccMapDFS(neighbor, edges, sccNumber, sccMap)

def computeReversedPostOrder(edges):
    n = len(edges)
    visited = [False] * n
    postOrder = []
    for i in range(n):
        if not visited[i]:
            postOrderDFS(i, edges, visited, postOrder)
    return postOrder[::-1]

def postOrderDFS(node, edges, visited, postOrder):
    visited[node] = True
    for neighbor in edges[node]:
        if not visited[neighbor]:
            postOrderDFS(neighbor, edges, visited, postOrder)
    postOrder.append(node)

In [6]:
airports = ["BGI", "CDG", "DEL", "DOH", "DSM", "EWR", "EYW", "HND", "ICN", "JFK", "LGA", "LHR", "ORD", "SAN", "SFO", "SIN", "TLV", "BUD"]

routes = [
    ["DSM", "ORD"],
    ["ORD", "BGI"],
    ["BGI", "LGA"],
    ["SIN", "CDG"],
    ["CDG", "SIN"],
    ["CDG", "BUD"],
    ["DEL", "DOH"],
    ["DEL", "CDG"],
    ["TLV", "DEL"],
    ["EWR", "HND"],
    ["HND", "ICN"],
    ["HND", "JFK"],
    ["ICN", "JFK"],
    ["JFK", "LGA"],
    ["EYW", "LHR"],
    ["LHR", "SFO"],
    ["SFO", "SAN"],
    ["SFO", "DSM"],
    ["SAN", "EYW"]
]

startingAirport = "LGA"

airportConnections(airports, routes, startingAirport)

3

In [14]:
edges, airportIndexMap = buildAdjacencyList(airports, routes)
reverseEdges(edges)
reversedPostOrderOnReversedGraph = computeReversedPostOrder(edges)
computeSCCMap(reversedPostOrderOnReversedGraph, edges)

([1, 0, 0, 0, 1, 2, 1, 2, 2, 2, 1, 1, 1, 1, 1, 0, 0, 0], 3)